In [1]:
import pandas as pd
df_train = pd.read_csv('../data/embeddings/fasttext_20_newsgroups_train_embeddings.csv')
df_test = pd.read_csv('../data/embeddings/fasttext_20_newsgroups_test_embeddings.csv')

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class TextCNN(nn.Module):
    def __init__(self, input_dim=300, num_classes=10, dropout_prob=0.5):
        super(TextCNN, self).__init__()

        # Define the Convolutional Layers
        self.conv1 = nn.Conv1d(in_channels=input_dim, out_channels=128, kernel_size=5)
        self.conv2 = nn.Conv1d(in_channels=128, out_channels=128, kernel_size=5)
        self.conv3 = nn.Conv1d(in_channels=128, out_channels=128, kernel_size=5)

        # Define the Max Pooling Layer
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)

        # Define the Dropout Layer
        self.dropout = nn.Dropout(p=dropout_prob)

        # Define the Fully Connected (Output) Layer
        self.fc = nn.Linear(128, num_classes)

    def forward(self, x):
        # Forward pass through Conv1
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        x = self.dropout(x)

        # Forward pass through Conv2
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)
        x = self.dropout(x)

        # Forward pass through Conv3
        x = self.conv3(x)
        x = F.relu(x)
        x = self.pool(x)
        x = self.dropout(x)

        # Global average pooling (adaptive pooling to 1)
        x = F.adaptive_avg_pool1d(x, 1).squeeze(2)

        # Fully connected layer
        x = self.fc(x)
        
        return x

In [17]:
from torch.utils.data import DataLoader, TensorDataset
import json
import numpy as np
# Example dataset (replace with your own data)
# Assuming X_train is of shape [num_samples, input_dim, max_len]
# and y_train is of shape [num_samples]
X_train = np.stack(np.array(df_train["embeddings"].apply(json.loads)))
y_train = np.array(df_train["label"])

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

X_test = np.stack(np.array(df_test["embeddings"].apply(json.loads)))
y_test = np.array(df_test["label"])

y_test_tensor = torch.tensor(y_test, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)

test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

In [20]:
import torch.optim as optim


# Hyperparameters
epochs = 20
learning_rate = 0.001
dropout_prob = 0.5

# Initialize the model, criterion, and optimizer
model = TextCNN(input_dim=300, num_classes=20, dropout_prob=dropout_prob)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


# Training loop
for epoch in range(epochs):
    model.train()
    total_loss = 0

    for batch in train_loader:
        inputs, labels = batch
        print("Shape: ",inputs.shape)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}")

print("Training complete!")

Shape:  torch.Size([128, 300])


RuntimeError: Given groups=1, weight of size [128, 300, 5], expected input[1, 128, 300] to have 300 channels, but got 128 channels instead

In [15]:
X_test = torch.rand(1000, 300, 1000)
y_test= torch.randint(0, 10, (1000,))
test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=True)

In [16]:
y_test.shape

torch.Size([1000])